# Distribuição de temperatura em eletrodo ao longo de seu comprimento energizado na soldagem subaquática com arame tubular.

# Abordagem matemática

O modelo original do problema tem a forma:

$$
\frac{d}{dz} \left( k \frac{dT}{dz}\right) + w \rho \frac{d (c_v T)}{dz} - h(T - T_\infty) = -j^2 r
$$



Considerando $k$ e $c_v$ determinadas iterativamente, e a função $v = {dT}/{dz}$, pode-se escrever a equação na forma dada por:

$$
\frac{dT}{dz} = v
$$
$$
\frac{dv}{dz} = \frac{1}{k} \left( - w \rho c_v v + h(T - T_\infty) -j^2 r \right)
$$

## Teste do modelo computacional

Para uso do modelo computacional, em primeiro lugar deve-se fazer a importação das bibliotecas necessárias para lidar com os dados.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

O modelo da equação diferencial está implementado no módulo na função `edo_marcelo` do módulo `edo_marcelo`.

In [2]:
from marcelo_solver.edo_marcelo import edo_marcelo

A documentação da função pode ser consultada da seguinte forma. 

In [3]:
help(edo_marcelo)

Help on function edo_marcelo in module marcelo_solver.edo_marcelo:

edo_marcelo(parm, par_cond_cont, interv, method='RK45', f_k=<function cfcond_termica at 0x7f59e034fa60>, f_cv=<function calesp_vol at 0x7f59e141dc10>, f_rho=<function densidade at 0x7f59e03a9f70>)
    Recebe:
    
    · parm:tuple (
        coef_w:float,
        dic_h:dict, # ATENÇÃO AO TIPO DISTOANTE
        coef_Tinf:float, 
        coef_j:float, 
        coef_r:float,
        coef_delh:float,
        coef_V: float,
    ) - uma tupla de 7 parâmetros do tipo float para o modelo.
    
    · cond_cont:tuple (
        [cd00:float,cd01:float]:list, 
        [cd10:float,cd11:float]:list
    ) - uma tupla de duas listas com pares ordenados descritores das 
    condições de contorno. 
    
    · interv: tuple (if:float, sup:float, disc:float) - tupla contendo, 
    os limites inferior e superior de integração e o a discretização do 
    vetor.
    
    Por padrão a função usa Runge-Kutta de ordem 5(4) como método numérico.
 

Os parâmetros de execução do modelo são extraídos do arquivo `parametros.json` que deve sempre estar contido no diretório de execução do modelo. Sua estrutura é:

``` json
{
    "coef_w": "1.",
    "dic_h": {
        "1.": ["1.","1."],
        "115": ["1.", "1."]
    },
    "coef_Tinf": "1.",
    "coef_j": "1,",
    "coef_r": "1.",
    "coef_variacao da entalpia": "120.",
    "coef_tensao": "20",
    "intervalo de integração": ["1.","1."],
    "discretização do intervalo": "1",
    "condicoes de contorno": {
        "extremidade consumida": ["1.", "1."],
        "bico de contato": ["1.", "1."]
    }
}
```

Por padrão os coeficientes $w$, $h$, $T_\infty$, $j$ e $r$, são configurados como unitários, o usuário deve fazer a modificação antes da execução, note que o parâmetro `dic_h` está registrado no segundo campo do `json`.

Para realizar a extração do parâmetros e posterior passagem à função `edo_marcelo` a função `carrega_parametros` deve ser importada do módulo `diversos`.

In [4]:
from marcelo_solver.diversos import carrega_parametros

Em seguida pode ser usada para o carregamento dos valores na forma de um dicionário Python.

In [5]:
dict_parametros = carrega_parametros()
dict_parametros

{'coef_w': '75.',
 'dic_h': {'1175.': ['0.', '9.'], '115': ['9.', '15.']},
 'coef_Tinf': '25.',
 'coef_j': '99.47',
 'coef_r': '5.62e-6',
 'coef_variacao da entalpia': '120.',
 'coef_tensao': '20',
 'intervalo de integração': ['0.', '15.'],
 'discretização do intervalo': '1.e-3',
 'condicoes de contorno': {'extremidade consumida': ['0.', '1515.'],
  'bico de contato': ['15.', '158.']}}

As aspas em torno dos valores numéricos indicam que estão armazenados na forma de `string` a mudança de tipo para ponto flutuante deve ser realizada explicitamente antes do uso, ou uma rotina para fazê-lo no próprio dicionário deve ser implementada.

## Execução do modelo

Na documentação da função `edo_marcelo`, mostrada acima, é possível ver que a função recebe três parâmetros:
1. `parm` - tupla com os coeficientes.
2. `cond_cont` - tupla com as condições de contorno
3. `interv` - tupla com o intervalo de integração e a discretização máxima desejada.

Uma tupla do Python é um contâiner de valores imutável, por isso de maior desempenho computacional. Eles podem ser criados da seguinte forma:

In [6]:
parm = [float(dict_parametros[k]) for k in dict_parametros.keys() if "coef" in k]
parm.insert(1, dict_parametros["dic_h"])
parm = tuple(parm)
parm

(75.0,
 {'1175.': ['0.', '9.'], '115': ['9.', '15.']},
 25.0,
 99.47,
 5.62e-06,
 120.0,
 20.0)

In [7]:
cond_cont = (
    [
        float(dict_parametros["condicoes de contorno"]\
              ["extremidade consumida"][0]),
        float(dict_parametros["condicoes de contorno"]\
              ["extremidade consumida"][1])
    ],
    [
        float(dict_parametros["condicoes de contorno"]\
              ["bico de contato"][0]),
        float(dict_parametros["condicoes de contorno"]\
              ["bico de contato"][1])
    ]
)
cond_cont

([0.0, 1515.0], [15.0, 158.0])

In [8]:
interv = (
    float(
        dict_parametros["intervalo de integração"][0]
    ),
    float(
        dict_parametros["intervalo de integração"][1]
    ),
    float(
        dict_parametros["discretização do intervalo"]
    )
)
interv

(0.0, 15.0, 0.001)

Agora os parâmetros podem ser passados para o modelo.

In [9]:
sol0 = edo_marcelo(parm, cond_cont, interv)

ValueError: ERRO!
A temperatura 1515.0000075356695 não é definida no intervalo de temperaturas do modelo.